In [1]:
import pandas as pd
from typing import Dict
import json
import urllib
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
) 
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup

import random
import os
import numpy as np

E:\anaconda3\envs\dacon_torch\lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
E:\anaconda3\envs\dacon_torch\lib\site-packages\torchvision\transforms\_transforms_video.py:25: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(41) # Seed 고정

In [3]:
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 1
frames_per_second = 30
slowfast_alpha = 4
num_clips = 10
num_crops = 3


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
model.blocks[6].proj.out_features = 5
model.to(device)

Using cache found in C:\Users\piai/.cache\torch\hub\facebookresearch_pytorchvideo_main


Net(
  (blocks): ModuleList(
    (0): MultiPathWayWithFuse(
      (multipathway_blocks): ModuleList(
        (0): ResNetBasicStem(
          (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
          (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
          (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
        )
        (1): ResNetBasicStem(
          (conv): Conv3d(3, 8, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
          (norm): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
          (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
        )
      )
      (multipathway_fusion): FuseFastToSlow(
        (conv_fast_to_slow): Conv3d(8, 16, kernel_size=(7, 1, 1), st

In [6]:
class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors. 
    """
    def __init__(self):
        super().__init__()
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list
    

In [7]:
class CustomDataset(Dataset):
    def __init__(self, file,device,transform=None, train=True):
        super().__init__()
        self.file = file
        self.len = len(self.file)
        self.device = device
        self.transform = transform
        self.train = train
        self.datalayer = PackPathway()
    
    def __getitem__(self, idx):
        if self.train :
            path = self.file[idx][0]
            label = self.file[idx][1]
            video = EncodedVideo.from_path(path)
            video_data = video.get_clip(start_sec=0, end_sec=1)
            video_data = self.transform(video_data)
            inputs = video_data["video"]
            inputs = [i.to(device) for i in inputs]

            return inputs, label
        else :
            path = self.file[idx]
            video = EncodedVideo.from_path(path)
            video_data = video.get_clip(start_sec=0, end_sec=1)
            video_data = self.transform(video_data)
            inputs = video_data["video"]
            inputs = [i.to(device) for i in inputs]

            return inputs
            

    def __len__(self):
        return self.len

In [8]:
transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

In [9]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train_data = []
test_data = []

for vid, path, label in train.values :
    train_data.append((path, label))

for vid, path in test.values :
    test_data.append(path)
    
train_dataset = CustomDataset(train_data,device,transform,train=True)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=False)

test_dataset = CustomDataset(test_data,device,transform,train=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [10]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(
        nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(
        nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters,
                    lr=5e-5, correct_bias=False)
criterion = nn.CrossEntropyLoss()

E:\anaconda3\envs\dacon_torch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


------------TRAIN------------
EPOCH: 1
train_loss:0.644475
------------TRAIN------------
EPOCH: 2
train_loss:0.056496
------------TRAIN------------
EPOCH: 3
train_loss:0.007663
------------TRAIN------------
EPOCH: 4
train_loss:0.002903
------------TRAIN------------
EPOCH: 5
train_loss:0.001048
------------TRAIN------------
EPOCH: 6
train_loss:0.000743
------------TRAIN------------
EPOCH: 7
train_loss:0.000578
------------TRAIN------------
EPOCH: 8
train_loss:0.000475
------------TRAIN------------
EPOCH: 9
train_loss:0.000393
------------TRAIN------------
EPOCH: 10
train_loss:0.000321
------------TRAIN------------
EPOCH: 11
train_loss:0.000300
------------TRAIN------------
EPOCH: 12
train_loss:0.000257
------------TRAIN------------
EPOCH: 13
train_loss:0.000215
------------TRAIN------------
EPOCH: 14
train_loss:0.000191
------------TRAIN------------
EPOCH: 15
train_loss:0.000160
------------TRAIN------------
EPOCH: 16
train_loss:0.000148
------------TRAIN------------
EPOCH: 17
train_los

In [12]:
torch.save(model.state_dict(), 'set_seed_slowfast.pth')

In [13]:
model.load_state_dict(torch.load('set_seed_slowfast.pth'))

<All keys matched successfully>

In [15]:
total_pred = np.zeros(0)
model.eval()
with torch.no_grad():
    for idx, d in enumerate(test_loader):
        output = model(d)
        
        pred = output.argmax(dim=1, keepdim=True)
        
        total_pred = np.append(total_pred, pred.cpu().numpy())

In [16]:
total_pred

array([1., 3., 0., 2., 4., 2., 4., 3., 1., 2., 2., 2., 0., 4., 1., 4., 3.,
       1., 4., 1., 2., 2., 2., 3., 2., 3., 1., 4., 4., 1., 3., 1., 0., 4.,
       3., 4., 2., 3., 2., 0., 2., 4., 4., 3., 3., 4., 4., 0., 4., 1., 3.,
       4., 4., 0., 1., 3., 4., 2., 0., 4., 0., 3., 4., 2., 4., 2., 1., 0.,
       2., 4., 3., 1., 3., 3., 3., 0., 4., 4., 3., 0., 1., 4., 2., 1., 3.,
       2., 4., 3., 0., 2., 1., 2., 2., 0., 4., 1., 4., 0., 2., 3., 2., 4.,
       1., 4., 0., 2., 0., 0., 3., 1., 0., 1., 4., 3., 1., 1., 2., 1., 1.,
       3., 4., 3., 1., 1., 3., 4., 0., 2., 1., 4., 0., 0., 4., 2., 4., 4.,
       0., 2., 2., 0., 0., 0., 4., 1., 3., 1., 2., 2., 2., 4., 0., 1., 2.])

In [17]:
submit = pd.read_csv('./sample_submission.csv')
submit

,id,label
0,TEST_000,0
1,TEST_001,0
2,TEST_002,0
3,TEST_003,0
4,TEST_004,0
...,...,...
148,TEST_148,0
149,TEST_149,0
150,TEST_150,0
151,TEST_151,0


In [18]:
submit['label'] = total_pred
submit.head()

,id,label
0,TEST_000,1.0
1,TEST_001,3.0
2,TEST_002,0.0
3,TEST_003,2.0
4,TEST_004,4.0


In [19]:
submit.to_csv('set_seed_slowfast_submit.csv', index=False)